In [1]:
import os
from os import path
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from utils import *

In [2]:
out_path = '../hint/out/'
data_path = '../data/'
data_files = 'pancancer/'
files = glob(data_path+data_files+'*/[!MANIFEST]*.txt')

# Fix the genes in expression data

### You can skip unitl the last two cells in this section,  and load final_genes and hint_u_pan

In [25]:
df_genename = pd.read_csv('../data/genenames_2019_01_17.txt', delimiter = '\t')
df_genename = df_genename[df_genename['NCBI Gene ID'].notnull()]
df_genename['ID'] = df_genename['NCBI Gene ID'].apply(lambda x: int(x))
df_genename = df_genename.drop(['HGNC ID', 'NCBI Gene ID'], axis= 1)

In [212]:
id2gene = {id_:g for id_,g in zip(df_genename.ID.values, df_genename['Approved symbol'].values)}
gene2id = {g:id_ for id_,g in zip(df_genename.ID.values, df_genename['Approved symbol'].values)}    

In [28]:
with open('../data/genes_pancancer_all_subtypes.txt', 'r') as f:
    genes_in_dataset= [l.rstrip() for l in f.readlines()]

In [168]:
genes_in_hint = []
with open('../hint/data/hint_index_file.txt', 'r') as f:
    for l in f.readlines():
        genes_in_hint.append(l.rstrip().split()[1])

In [ ]:
# Run this to get gene_and_id

# df = pd.read_csv(files[0], delimiter = '\t', low_memory=False)
# df.drop(df.index[0], inplace=True)
# gene_and_id = df.iloc[:, 0].values

In [227]:
# if an error check the cell above

exp_name2id = {}
exp_id2name = {}
for g in gene_and_id:
    name,id_ = g.split('|')
    exp_name2id[name] = id_
    exp_id2name[id_] = name      

In [228]:
missing_genes = []
for gene in genes_in_hint :
    if not gene in exp_name2id.keys():
        missing_genes.append(gene)

In [223]:
len(missing_genes)

432

In [229]:
fixed_genes = 0
for g in missing_genes:
    if g in gene2id.keys():
        id_ = gene2id[g]
        if str(id_) in exp_id2name.keys():
            fixed_genes += 1
            exp_id2name[str(id_)] = g

In [226]:
fixed_genes

152

In [231]:
final_genes = []
for g in gene_and_id:
    name,id_ = g.split('|')
    final_genes.append(exp_id2name[id_])

In [240]:
# Gene names used for indexing the expression data 

# with open('../data/exp_final_gene_list.txt', 'w') as f:
#     f.write('\n'.join(final_genes))


with open('../data/exp_final_gene_list.txt', 'r') as f:
    final_genes= [l.rstrip() for l in f.readlines()]

In [241]:
# Genes in hint and in expression data

# with open('../data/hint_inters_pan.txt', 'w') as f:
#     f.write('\n'.join(hint_u_pan))
    
with open('../data/hint_inters_pan.txt', 'r') as f:
    hint_u_pan= [l.rstrip() for l in f.readlines()]

In [284]:
hint_u_pan = set(genes_in_hint).intersection(set(final_genes))
len(hint_u_pan)

9523

In [291]:
print('genes in hit not in expression data is {}'.format(len(set(genes_in_hint)) - len(hint_u_pan)))

genes in in not in expression data is 339


# Merge the data

In [3]:
allowed_cancer_types = ['LUSC','HNSC','BLCA', 'GBM', 'LUAD', 'KIRC', 'UCEC', 'COADREAD', 'BRCA']
all_types = [c.split('/')[-1].split('.')[0] for c in files]
cancer_type = [c for c in all_types if c in allowed_cancer_types]

In [4]:
cancer_type

['LUSC', 'HNSC', 'BLCA', 'GBM', 'LUAD', 'KIRC', 'UCEC', 'COADREAD', 'BRCA']

In [273]:
df = pd.read_csv(files[0], delimiter = '\t', low_memory=False)
df.drop(df.index[0], inplace=True)
gene_and_id = df.iloc[:, 0].values
df.index = final_genes
df.drop(df.columns[0], axis=1, inplace= True)
df.index.names = ['genes']
df = df.loc[list(hint_u_pan)]

In [235]:
hint_u_pan = set(genes_in_hint).intersection(set(final_genes))

In [238]:
df2.shape

(9523, 552)

In [237]:
df2.head()

TCGA-18-3406-01A-01R-0980-07 TCGA-18-3407-01A-01R-0980-07  \
genes                                                               
NUP98                      1388.1890                    3714.0818   
ZKSCAN5                     241.7323                     390.5804   
LRRC7                         0.7874                       0.9515   
RHO                           0.0000                       0.0000   
WASH3P                      133.8583                     283.0637   

        TCGA-18-3408-01A-01R-0980-07 TCGA-18-3409-01A-01R-0980-07  \
genes                                                               
NUP98                      1574.9666                    2000.0000   
ZKSCAN5                     389.1443                     428.7179   
LRRC7                        14.8302                       0.5128   
RHO                           0.0000                       0.5128   
WASH3P                       58.7276                     221.0256   

        TCGA-18-3410-01A-01R-0980-07 TCGA-18-3411-01A-01R-0980-07  \
genes                                                               
NUP98                      1679.1381                    2547.7220   
ZKSCAN5                     359.2245                     411.9481   
LRRC7                         1.5755                       0.0000   
RHO                           1.5755                       0.0000   
WASH3P                      290.6882                     159.3081   

        TCGA-18-3412-01A-01R-0980-07 TCGA-18-3414-01A-01R-0980-07  \
genes                                                               
NUP98                      1995.6964                    1415.9129   
ZKSCAN5                     344.2879                     428.6505   
LRRC7                         0.0000                       4.4305   
RHO                           0.0000                       0.3692   
WASH3P                      333.7246                     372.9001   

        TCGA-18-3415-01A-01R-0980-07 TCGA-18-3416-01A-01R-0980-07  \
genes                                                               
NUP98                      2402.3121                    2099.3817   
ZKSCAN5                     306.9364                     348.1303   
LRRC7                         0.5780                       0.0000   
RHO                           0.0000                       0.0000   
WASH3P                      127.1676                     242.1291   

                    ...              TCGA-NK-A5CX-01A-11R-A26W-07  \
genes               ...                                             
NUP98               ...                                 2080.4499   
ZKSCAN5             ...                                  285.1204   
LRRC7               ...                                    0.9781   
RHO                 ...                                    0.4891   
WASH3P              ...                                  553.1239   

        TCGA-NK-A5D1-01A-11R-A26W-07 TCGA-NK-A7XE-01A-12R-A405-07  \
genes                                                               
NUP98                      2058.9953                    1773.7042   
ZKSCAN5                     441.5888                     200.3793   
LRRC7                         1.7523                       2.5284   
RHO                           0.0000                       0.6321   
WASH3P                      452.6869                     790.7712   

        TCGA-O2-A52N-01A-11R-A26W-07 TCGA-O2-A52Q-01A-11R-A26W-07  \
genes                                                               
NUP98                      3149.7268                    1639.2669   
ZKSCAN5                     209.8361                     278.8444   
LRRC7                         0.0000                       0.4970   
RHO                           0.5464                       0.4970   
WASH3P                      730.0546                     481.1432   

        TCGA-O2-A52S-01A-11R-A262-07 TCGA-O2-A52V-01A-31R-A262-07  \
genes                                                              

In [274]:
# Merge all the datasets that are part of cancer type list
patients_list = []
patients_list.append(df.columns.values[1:])
for file in files[1:]:
    
    type_ = file.split('/')[-1].split('.')[0] 
    if not type_ in cancer_type: 
        print('Skipped cancer ', type_)
        continue
        
    df1 = pd.read_csv(file, delimiter = '\t', low_memory=False)
    df1.drop(df1.index[0], inplace=True)
    df1.index = final_genes
    df1 = df1.loc[list(hint_u_pan)]
    patients_list.append(df1.columns.values[1:])
    
    df1.drop(df1.columns[0], axis=1 ,inplace=True)
    
    df = df.join(df1)

Skipped cancer  LAML
Skipped cancer  OV


In [275]:
df_ = df.T

In [276]:
df_.index.names = ['genes']

In [281]:
df_.columns.name = 'patients'

In [287]:
df_['y'] = [0 if int(str(p).split('-')[3][:2]) > 10 else 1 for p in df_.index.values]

In [289]:
np.unique(df_['y'], return_counts = True)

(array([0, 1]), array([ 437, 4307]))

In [292]:
df_.to_csv(data_path+'pancancer_all_subtypes.csv')

#### Data is ready... Have fun

In [3]:
df = pd.read_csv(data_path+'pancancer_all_subtypes.csv')

In [ ]:
components_path = '../hint/out/connectrd_components_isolarge_v2_n2500/'
module_path = components_path + '{}/'

In [ ]:
def get_n100_file(module_path):
    return glob(module_path+'cc_n100_*.txt')

In [4]:
random_state = 1234
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score